# AI & Security Project

**regular_models.ipynb**: in this notebook we try out several image classification models on our dataset 'TinyImageNet'. We use an **extreme** amount of different configurations


## Step 0: Configurations


In [53]:
import warnings

warnings.filterwarnings("ignore")

## Step 1. dataset and libraries


In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import numpy as np


data_directory = r"./data/TinyImageNet/tiny-imagenet-200/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
import os
import requests
import zipfile
import shutil

def download_and_prepare_tinyimagenet(data_dir="tiny-imagenet"):
	"""
	Download and extract the TinyImageNet dataset, and prepare it for PyTorch's ImageFolder.
	"""
	url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
	zip_file = "tiny-imagenet-200.zip"
	extract_dir = "tiny-imagenet-200"
	
	# Create the directory if it doesn't exist
	if not os.path.exists(data_dir):
		os.makedirs(data_dir)
	
	# Download the dataset
	print("Downloading TinyImageNet dataset...")
	response = requests.get(url, stream=True)
	with open(os.path.join(data_dir, zip_file), "wb") as f:
		shutil.copyfileobj(response.raw, f)
	print("Download complete.")
	
	# Extract the dataset
	print("Extracting dataset...")
	with zipfile.ZipFile(os.path.join(data_dir, zip_file), "r") as zip_ref:
		zip_ref.extractall(data_dir)
	print("Extraction complete.")
	
	# Organize validation dataset
	val_dir = os.path.join(data_dir, extract_dir, "val")
	val_images_dir = os.path.join(val_dir, "images")
	val_annotations = os.path.join(val_dir, "val_annotations.txt")
	
	# Parse the validation annotations
	print("Organizing validation dataset...")
	with open(val_annotations, "r") as f:
		lines = f.readlines()
	
	val_labels = {}
	for line in lines:
		parts = line.split("\t")
		val_labels[parts[0]] = parts[1]
	
	# Create subdirectories for validation classes
	for label in set(val_labels.values()):
		os.makedirs(os.path.join(val_dir, label), exist_ok=True)
	
	# Move validation images to corresponding class subdirectories
	for img, label in val_labels.items():
		src_path = os.path.join(val_images_dir, img)
		dst_path = os.path.join(val_dir, label, img)
		shutil.move(src_path, dst_path)
	
	# Remove the original images directory and annotation file
	shutil.rmtree(val_images_dir)
	os.remove(val_annotations)
	
	print("Dataset is ready at:", os.path.join(data_dir, extract_dir))
	return os.path.join(data_dir, extract_dir)


# Check if the dataset is already downloaded, by checking is /train/, /val/ and /test/ directories are present
downloaded = os.path.exists(data_directory + "train/")\
    and os.path.exists(data_directory + "test/")\
    and os.path.exists(data_directory + "val/")


if not downloaded:
	dataset_path = download_and_prepare_tinyimagenet(data_directory)
	print("TinyImageNet dataset is available at:", dataset_path)

# This took 3m30 on my machine

## Step 2: models

In [56]:
from torchvision.models import ResNet18_Weights, ResNet34_Weights, \
								ResNet50_Weights, ResNet101_Weights, \
								ResNet152_Weights


# Model definitions
model_definitions = {
	"ResNet18": lambda: models.resnet18(weights=ResNet18_Weights.DEFAULT),
	"ResNet50": lambda: models.resnet50(weights=ResNet50_Weights.DEFAULT),
	"ResNet152": lambda: models.resnet152(weights=ResNet152_Weights.DEFAULT),
}

# Model configurations (hyperparameters)
model_configs = {
    
    # Configurations with learning rate 0.1
    "Config-1A-0.3WD": {"lr": 0.1, "optimizer": "SGD", "weight_decay": 0.3},
    "Config-1A-0.9WD": {"lr": 0.1, "optimizer": "SGD", "weight_decay": 0.9},
    "Config-1B-0.3WD": {"lr": 0.1, "optimizer": "AdamW", "weight_decay": 0.3},
    "Config-1B-0.9WD": {"lr": 0.1, "optimizer": "AdamW", "weight_decay": 0.9},
    
    # Configurations with learning rate 0.01
    "Config-2A-0.3WD": {"lr": 0.01, "optimizer": "SGD", "weight_decay": 0.3},
    "Config-2A-0.9WD": {"lr": 0.01, "optimizer": "SGD", "weight_decay": 0.9},
    "Config-2B-0.3WD": {"lr": 0.01, "optimizer": "AdamW", "weight_decay": 0.3},
    "Config-2B-0.9WD": {"lr": 0.01, "optimizer": "AdamW", "weight_decay": 0.9},
    
    # Configurations with learning rate 0.001
    "Config-3A-0.3WD": {"lr": 0.001, "optimizer": "SGD", "weight_decay": 0.3},
    "Config-3A-0.9WD": {"lr": 0.001, "optimizer": "SGD", "weight_decay": 0.9},
    "Config-3B-0.3WD": {"lr": 0.001, "optimizer": "AdamW", "weight_decay": 0.3},
    "Config-3B-0.9WD": {"lr": 0.001, "optimizer": "AdamW", "weight_decay": 0.9},
    
    # Configurations with learning rate 0.0001
    "Config-4A-0.3WD": {"lr": 0.0001, "optimizer": "SGD", "weight_decay": 0.3},
    "Config-4A-0.9WD": {"lr": 0.0001, "optimizer": "SGD", "weight_decay": 0.9},
    "Config-4B-0.3WD": {"lr": 0.0001, "optimizer": "AdamW", "weight_decay": 0.3},
    "Config-4B-0.9WD": {"lr": 0.0001, "optimizer": "AdamW", "weight_decay": 0.9},
}

# General hyperparameters
hyperparameters = {
	'batch_size': 64,
	'epochs': 20,
	'num_workers': 4,
}

## Step 3: set-up training loop

In [57]:
# Step 1: Define TinyImageNet Dataset
def get_tinyimagenet_loaders(data_dir, batch_size, num_workers=hyperparameters['num_workers']):
	"""
	Set up data loaders for TinyImageNet dataset.
	"""
	transform_train = transforms.Compose([
		transforms.RandomCrop(64, padding=4),
		transforms.RandomHorizontalFlip(),
		transforms.ToTensor(),
		transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262])
	])

	transform_val = transforms.Compose([
		transforms.ToTensor(),
		transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262])
	])

	train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform_train)
	val_dataset = datasets.ImageFolder(root=f"{data_dir}/val", transform=transform_val)

	train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
	val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

	return train_loader, val_loader

# Step 2: Define Training and Evaluation Functions
def train_one_epoch(model, loader, criterion, optimizer, device):
	"""
	Train the model for one epoch.
	"""
	model.train()
	running_loss = 0.0
	correct = 0
	total = 0

	for inputs, targets in tqdm(loader, desc="	Training", leave=False):
	# for inputs, targets in loader:
		inputs, targets = inputs.to(device), targets.to(device)
		optimizer.zero_grad()

		outputs = model(inputs)
		loss = criterion(outputs, targets)
		loss.backward()
		optimizer.step()

		running_loss += loss.item() * inputs.size(0)
		_, predicted = outputs.max(1)
		correct += predicted.eq(targets).sum().item()
		total += targets.size(0)

	epoch_loss = running_loss / total
	accuracy = correct / total
	return epoch_loss, accuracy

def evaluate(model, loader, criterion, device):
	"""
	Evaluate the model on validation data and compute top-1 and top-5 errors.
	"""
	model.eval()
	running_loss = 0.0
	top1_correct = 0
	total = 0
	top5_correct = 0

	with torch.no_grad():
		for inputs, targets in tqdm(loader, desc="	Validation", leave=False):
		# for inputs, targets in loader:
			inputs, targets = inputs.to(device), targets.to(device)

			outputs = model(inputs)
			loss = criterion(outputs, targets)
			running_loss += loss.item() * inputs.size(0)

			_, top1_preds = outputs.max(1)
			top1_correct += top1_preds.eq(targets).sum().item()

			_, top5_preds = outputs.topk(5, 1, True, True)
			top5_correct += sum([targets[i] in top5_preds[i] for i in range(targets.size(0))])
			total += targets.size(0)

	epoch_loss = running_loss / total
	top1_error = 1 - (top1_correct / total)
	top5_error = 1 - (top5_correct / total)

	return epoch_loss, top1_error, top5_error

# Step 3: Define Model Training and Comparison
def compare_models(models, model_configs, hyperparameters, data_dir, num_epochs, device="cuda"):
	"""
	Fine-tune and evaluate multiple models with different configurations.
	"""
	results = []

	train_loader, val_loader = get_tinyimagenet_loaders(data_dir, batch_size=hyperparameters['batch_size'])

	for model_name, model_fn in models.items():
		for config_name, config in model_configs.items():
			print(f"Training {model_name} with configuration {config_name}:")
			model = model_fn().to(device)

			# Update only classifier layer for fine-tuning
			for param in model.parameters():
				param.requires_grad = False

			# Fine-tune the last layer
			if hasattr(model, 'fc'):  # For ResNet models
				model.fc = nn.Linear(model.fc.in_features, 200).to(device)
			elif hasattr(model, 'classifier'):  # For models like VGG or AlexNet
				model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 200).to(device)

			for param in model.fc.parameters():
				param.requires_grad = True

			# Select optimizer based on config
			if config['optimizer'] == 'SGD':
				optimizer = optim.SGD(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
			elif config['optimizer'] == 'AdamW':
				optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

			criterion = nn.CrossEntropyLoss()

			for epoch in range(num_epochs):
				print(f"	Epoch {epoch + 1}/{num_epochs}")
				train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
				val_loss, top1_error, top5_error = evaluate(model, val_loader, criterion, device)    

				results.append({
					'model': model_name,
					'config': config_name,
					'epoch': epoch + 1,
					'train_loss': train_loss,
					'train_acc': train_acc,
					'val_loss': val_loss,
					'top1_error': top1_error,
					'top5_error': top5_error
				})

	return results


## Step 4: train and evaluate

In [ ]:
# Train and evaluate
results = compare_models(
	models=model_definitions,
	model_configs=model_configs,
	hyperparameters=hyperparameters,
	data_dir=data_directory,
	num_epochs=hyperparameters['epochs'],
	device=device
)

In [34]:
import time
import pandas as pd

# Export results to CSV
current_time = time.strftime("%Y%m%d-%H%M")
results_df = pd.DataFrame(results)
results_df.to_csv(f"./exports/data/regular_models_results_(many configurations)_{current_time}.csv", index=False)